# RAG bassato su notizie di giornali

Benvenuti in questo notebook dedicato all'esplorazione di un sistema **RAG (Retrieval-Augmented Generation)** arricchito con funzionalità di filtering dei metadati.

## Funzionalità Principali

Il sistema consente di affinare la ricerca dei documenti attraverso i seguenti filtri di metadati: 
- **Sezione**: specifica la sezione dell'articolo di giornale (es. politica, esteri, cronache, ecc.).
- **Data**: permette di selezionare articoli pubblicati in un determinato intervallo temporale.
- **Autore**: consente di focalizzarsi sugli articoli scritti da un autore specifico.

## Miglioramento dei Risultati

Per ottenere risposte ancora più pertinenti, puoi sfruttare tecniche di **prompt engineering** modificando i prompt che si trovano nel file `rag_utils`. Qui potrai ottimizzare i prompt utilizzati dal generatore LLM per adattarli al tuo caso d'uso specifico.

Inoltre, puoi regolare due parametri chiave del sistema:
- **K**: definisce il numero di documenti recuperati dal retriever nella fase iniziale.
- **top_n**: specifica quanti documenti, dopo la fase di reranking, vengono utilizzati come contesto per il modello generativo.

Questi parametri permettono di bilanciare precisione e recall, garantendo un controllo granulare sui risultati.

## Domanda di Esempio

> Di cosa parlano gli articoli della sezione politica che parlano di Toti?

Usando i filtri e i parametri descritti, potrai scoprire come configurare il sistema per rispondere a domande complesse come questa, migliorando continuamente la qualità delle risposte. 

---

Buona esplorazione!

In [1]:
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
)
import warnings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
import langchain_core
import logging

# Configura il livello di log
logging.basicConfig(level=logging.ERROR, format="%(levelname)s: %(message)s")
warnings.filterwarnings("ignore")

# Retriever

In [16]:
%load_ext autoreload
%autoreload 2
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt
)
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.query_constructors.opensearch import OpenSearchTranslator
from typing import Any, Dict, Callable, List, Optional
from langchain.schema import Document
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.runnables import RunnableConfig, ensure_config
import rag_utils
import time

''' ---------------- FILTER CORRECTION -------------------
Funzioni custom create per verificare la correttezza del filtro sui metadati creato dall'LLM:
nel caso in cui siano presenti filtri su campi inesistenti, questi verranno rimossi dal filtro per evitare di ottenere dei
risultati vuoti'''

def correct_structured_query(strucuterd_query):
    if isinstance(strucuterd_query, langchain_core.structured_query.StructuredQuery):
        if strucuterd_query.filter is not None:
            strucuterd_query.filter = correct_filter(strucuterd_query.filter)
    return strucuterd_query


def correct_filter(filtro):
    if isinstance(filtro, langchain_core.structured_query.Operation):
        for i in range(len(filtro.arguments) - 1, -1, -1):
            if isinstance(filtro.arguments[i], langchain_core.structured_query.Operation):
                filtro.arguments[i] = correct_filter(filtro.arguments[i])
            else:
                if filtro.arguments[i].attribute not in rag_utils.useful_attributes:
                    del filtro.arguments[i]
        if (
                filtro.operator == langchain_core.structured_query.Operator.AND or filtro.operator == langchain_core.structured_query.Operator.OR) and len(
                filtro.arguments) < 2:
            if len(filtro.arguments) == 1:
                filtro = filtro.arguments[0]
            else:
                filtro = None
    elif isinstance(filtro, langchain_core.structured_query.Comparison):
        if filtro.attribute not in rag_utils.useful_attributes:
            filtro = None
            print(f"Deleted, now: {filtro}")
    return filtro

''' ---------------- END FILTER CORRECTION ------------------- '''

class OpenSearchRetrieverWrapper:
    def __init__(self, llm_model_name:str, ollama_url:str, vectorstore:OpenSearchVectorSearch, 
                 top_n:int, search_kwargs: dict = {"k": 20},**query_model_kwargs:dict):
        constructor_prompt = get_query_constructor_prompt(
                rag_utils.document_content_description,
                rag_utils.metadata_field_info,
                allowed_comparators=rag_utils.allowed_comparators,
                examples=rag_utils.examples,
            )
        
        query_model = ChatOllama(
            model=llm_model_name, 
            base_url= ollama_url,
            temperature=query_model_kwargs.get("temperature", 0), #questo parametro rende l'LLM "meno creativo" nella fase di creazione del filtro
            streaming=query_model_kwargs.get("streaming", True),
            format='json', 
            seed=1   #puoi mettere il valore che vuoi, o eliminarlo
        )

        output_parser = StructuredQueryOutputParser.from_components(fix_invalid=True)
        self.constructor_prompt = constructor_prompt
        # la pipeline di creazione della query presenta anche la funzione custom di filter correction
        self.query_constructor = constructor_prompt | query_model | output_parser | correct_structured_query

        # custom in quando è necessario modificare leggermente la funzione _get_docs_with_query
        retriever = CustomSelfQueryRetriever(
            query_constructor=self.query_constructor,
            vectorstore=vectorstore,
            structured_query_translator=OpenSearchTranslator(),
            search_kwargs = search_kwargs,
            verbose=True
        )

        # reranker
        compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=top_n)
        self.retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )
        
class CustomSelfQueryRetriever(SelfQueryRetriever):    
    def _get_docs_with_query(self, query: str, search_kwargs: Dict[str, Any]) -> List[Document]:
        """Get docs, adding score information."""
        efficient_filter = search_kwargs.get("filter", None)
        if efficient_filter is not None:
            return self.vectorstore.similarity_search(query=query,k=search_kwargs["k"], search_type="approximate_search",
                                                      vector_field="embeddings",efficient_filter=efficient_filter)
        else:
            return self.vectorstore.similarity_search(query=query,k=search_kwargs["k"], search_type="approximate_search",
                                                      vector_field="embeddings")      

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants

In [3]:
OLLAMA_BASE_URL = ""        # dove è hostato Ollama
OPEN_SEARCH_URL = ""        # dove è ostato OpenSearch
# puoi usare gli embeddings che preferisci, l'importante che siano gli stessi usati in fase di memorizzazione
EMBEDDINGS_NAME = "intfloat/multilingual-e5-large"   
DEVICE_ID = 0
INDEX_NAME = ""             # nome dell'indice dove sono memorizzati i dati
TOP_N = 4       # più è grande, più aumenti il contesto dato in pasto all'LLM in fase di generazione. Fai attenzione al problema del Lost in the middle
K = 20          # più è grande, più sarà lento, ma aumenti la recall

In [4]:
from sentence_transformers import SentenceTransformer
embeddings =  HuggingFaceEmbeddings(model_name=EMBEDDINGS_NAME, model_kwargs={"trust_remote_code": True, "device": f"cuda:{DEVICE_ID}"}) 

In [5]:
docsearch = OpenSearchVectorSearch(
    index_name=INDEX_NAME,
    embedding_function=embeddings,
    opensearch_url=OPEN_SEARCH_URL,
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    
)

# Chain definition

In [6]:
%load_ext autoreload
%autoreload 2
def format_docs(docs):
        return "\n\n".join(f"[{index}] {doc.page_content}" for index,doc in enumerate(docs))

def get_chain(retriever, llm_generator_name, user_prompt):
    llm = ChatOllama(model=llm_generator_name, base_url=OLLAMA_BASE_URL,temperature=0, stream=True)
    
    prompt = ChatPromptTemplate.from_messages(
            [
                ('system',rag_utils.system_prompt),
                ('human', user_prompt)
            ])
    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )

    rag_chain_with_source = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)

    return rag_chain_with_source

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
LLM_QUERY_CREATOR = "qwen2.5:7b"    # puoi usare l'LLM che preferisci, ovviamente fai attenzione che il prompt sia ancora adatto

# Test

In [8]:
opensearch_retriever_wrapper = OpenSearchRetrieverWrapper(
    vectorstore=docsearch,
    ollama_url=OLLAMA_BASE_URL,
    llm_model_name=LLM_QUERY_CREATOR,
    query_constructor_llm_model_url=OLLAMA_BASE_URL,
    temperature=0,
    top_n=TOP_N,
    search_kwargs={
        "k": K
    }
)

In [ ]:
from datetime import datetime

# Definizione manuale dei mesi in italiano
mesi = [
    "Gennaio", "Febbraio", "Marzo", "Aprile", "Maggio", "Giugno",
    "Luglio", "Agosto", "Settembre", "Ottobre", "Novembre", "Dicembre"
]

# Ottieni la data odierna
oggi = datetime.now()

# Formatta la data
giorno = oggi.day
mese = mesi[oggi.month - 1]  # Mese in italiano
anno = oggi.year

# Componi la data
data_formattata = f"{giorno} {mese} {anno}"

In [9]:
question = "Di cosa parlano gli articoli della sezione politica che parlano di Toti?"

In [10]:
# Questo passaggio non è necessario, ho deciso io di impostare le domande sempre in questo modo. Puoi fare il prompt engeneering che vuoi
# fai attenzione agli esempi in rag_utils.py
question = f"Oggi è {data_formattata}. {question}"
question

'Oggi è 18 Dicembre 2024. Di cosa parlano gli articoli della sezione politica che parlano di Toti?'

## Filter creation
Vediamo come viene creato il filtro per recuperare i documenti rilevanti da parte dell'LLM

In [11]:
structured_query = opensearch_retriever_wrapper.query_constructor.invoke(question)
structured_query

StructuredQuery(query='Di cosa parlano gli articoli che parlano di Toti?', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='date', value={'date': '2024-12-18', 'type': 'date'}), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='section', value='politica')]), limit=None)

## Retriever
Vediamo quali sono i documenti ritrovati

In [12]:
context = opensearch_retriever_wrapper.retriever.invoke(question)
context

[Document(metadata={'id': '4acf1077-8f43-4d60-a480-062c7b8bad98', 'relevance_score': 0.0062182066, 'author': 'Claudio Bozza', 'date': '2024-12-18T11:46:55+0100', 'type': 'content', 'section': 'politica', 'url': 'https://www.corriere.it/politica/24_dicembre_18/valditara-fa-causa-a-lagioia-il-ministro-chiede-20-mila-euro-allo-scrittore-ecco-perche-4d6d16fa-cfc0-434b-a431-184bf2bddxlk.shtml'}, page_content="\nValditara, nelle settimane scorse, aveva avviato una causa civile anche contro lo scrittore e insegnante Christian Raimo, che in tv aveva definito\xa0«cialtrone» il ministro. L'ufficio scolastico regionale ha poi sospeso Raimo per tre mesi dalla cattedra, con decurtazione del 50% dello stipendio. Così, dopo questo nuovo procedimento, è riesplosa la bufera politica: «Chiediamo al ministro e alla premier Meloni se esista ancora in questo Paese la libertà di manifestare il proprio pensiero. Valditara - ha detto la leader del Pd, Elly Schlein - si sta infatti distinguendo per la serie di

## Entire chain
Testiamo l'intera catena

In [13]:
chain = get_chain(opensearch_retriever_wrapper.retriever, LLM_QUERY_CREATOR, rag_utils.user_prompt)

In [15]:
for chunk in chain.stream(question):
    q = chunk.get('question',None)
    if q is not None:
        continue
    c = chunk.get('context',None)
    if c is not None:
        #print(f"Context: {c}", end='', flush=True)
        continue
    print(chunk.get('answer',None), end='', flush=True)

Gli articoli [1] e [3] della sezione politica parlano del patteggiamento giudiziario dell'ex governatore della Regione Liguria, Giovanni Toti. Oggi, sette mesi dopo i suoi arresti domiciliari, Toti ha patteggiato due anni e tre mesi, convertiti in 1620 ore di lavori socialmente utili. L'avvocato Stefano Savi ha precisato che il patteggiamento prescinde sia da un accertamento della responsabilità che da una ammissione della stessa. Toti non potrà ricandidarsi nei prossimi due anni, ma l'avvocato non ha fornito ulteriori dettagli su questo aspetto.